# Example of Moment Conditions and the Basic GMM

The first few cells load functions and data. See further down for the estimation.

In [1]:
include("jlFiles/printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
xx = readdlm("Data/FFmFactorsPs.csv",',',header=true)   #start on line 2, column 1

xx  = xx[1]
ym = xx[:,1]         #[yearmonth]
x  = xx[:,2]         #excess market returns, in %
xx = nothing

T  = size(x,1)

388

## Point Estimates and Distribution

To estimate the mean of $x_{t}$, use the following moment condition

$$
\frac{1}{T}\sum\nolimits_{t=1}^{T}g_{t}=0 \ \text{ where } \ g_{t}=x_{t}-\mu.
$$

The distribution of the estimates is

$$
\sqrt{T}(\hat{\mu}-\mu_{0})\overset{d}{\rightarrow}N(0,V) 
\ \text{ where } \ V=\left(  D_{0}^{\prime}S_{0}^{-1}D_{0}\right)  ^{-1}
$$

Clearly, $D_{0}=-1$ and if data is iid then $S_{0}=\mathrm{Var}(g_{t})$.

In [3]:
muHat = mean(x,1)                     #it is clear that this is the GMM estimate

g    = x .- muHat                     #moment condition, .- 'broadcasts muHat' to x
gbar = mean(g,1)
printlnPs("Sample moment condition at estimate: ",gbar)

T   = size(g,1)
S   = var(x)
D   = -1
V   = inv(D'inv(S)*D)

printlnPs("\nmuHat Std(muHat): ",[muHat sqrt(V/T)])

Sample moment condition at estimate:     -0.000

muHat Std(muHat):      0.602     0.234


## Newey-West

To calculate the Newey-West covariance matrix, we first need the
autocovariance matrices $\Omega_{s}=\operatorname*{Cov}\left(
g_{t},g_{t-s}\right)  $, which is estimated as 
$$ 
\Sigma_{t=s+1}^{T} (g_{t}-\bar{g})(g_{t-s}-\bar{g})^{\prime}/T.
$$

Then we form a linear
combination (with tent-shaped weights) of those autocovariance matrices (from
lag $-m$ to $m$), or equivalently

$$
\widehat{\operatorname*{Cov}}\left(  \sqrt{T}\bar{g}\right)  =\Omega_{0}
+\sum_{s=1}^{m}\left(  1-\frac{s}{m+1}\right)  \left(  \Omega_{s}+\Omega
_{s}^{\prime}\right).
$$


In [4]:
function NWFn(g,m=0)
 #NWFn    Calculates covariance matrix of sqrt(T)*sample average.

  T = size(g,1)                     #g is Txq
  m = min(m,T-1)                    #number of lags

  g = g .- mean(g,1)                #Normalizing to Eg=0

  S = g'g/T                         #(qxT)*(Txq)
  for s = 1:m
    Gamma_s = g[s+1:T,:]'g[1:T-s,:]/T   #same as Sum[g(t)*g(t-s)',t=s+1,T]
    S       = S  +  ( 1 - s/(m+1) ) * (Gamma_s + Gamma_s')
  end

  return S

end

NWFn (generic function with 2 methods)

In [5]:
println("\nAs an alternative, use Newey-West for S")

Sb = NWFn(g,3)            #Newey-West covariance matrix with 3 lags

Vb = inv(D'inv(Sb)*D)

printlnPs("\nmuHat with Std(muHat) according to NW: ",[muHat sqrt.(Vb/T)])


As an alternative, use Newey-West for S

muHat with Std(muHat) according to NW:      0.602     0.246
